# prospect + specutils

Project: Convert DESI, SDSS & eBOSS spectra to specutils container objects, and modify prospect to accept those objects.

In [ ]:
import os
import numpy as np
from astropy.table import Table
from specutils import Spectrum1D, SpectrumCollection, SpectrumList
from prospect.specutils import read_spectra, read_spPlate, read_spZbest
from prospect.plotspecutils import plotspectra


## DESI spectra file

DESI spectra are stored by "channel" ("channel", "band" and "spectrograph arm" are used interchangably).  There are 10 spectrographs, each with three arms, called 'b', 'r' & 'z'.  In a DESI spectra file, all arms are grouped together, with a common wavelength solution for each arm, but the solutions do not overlap or have the same shape.  Thus we can't use a Spectrum1D or SpectrumCollection object, but we can use a SpectrumList containing three Spectrum1D objects.

In [ ]:
os.environ['DESI_SPECTRO_REDUX'] = os.path.join(os.environ['DESI_ROOT'], 'datachallenge', 'reference_runs', '19.12', 'spectro', 'redux')
os.environ['SPECPROD'] = 'mini'
nside = '64'
pixnum = '5305'
pixgroup = pixnum[0:2]
desi_spectra = os.path.join(os.environ['DESI_SPECTRO_REDUX'], os.environ['SPECPROD'],
                            f'spectra-{nside}', pixgroup, pixnum, f'spectra-{nside}-{pixnum}.fits')
desi_redshifts = os.path.join(os.environ['DESI_SPECTRO_REDUX'], os.environ['SPECPROD'],
                              f'spectra-{nside}', pixgroup, pixnum, f'zbest-{nside}-{pixnum}.fits')
print(desi_spectra)
print(desi_redshifts)

## SDSS/eBOSS spPlate file

SDSS spectra are stored per-plate in spPlate files.  These contain 640 spectra for the original SDSS spectrograph or 1000 spectra for the BOSS/eBOSS spectrograph.  All spPlate files have a common wavelength solution, so a spPlate file can be represented by a Spectrum1D object.

In [ ]:
os.environ['SPECTRO_REDUX'] = os.path.join(os.environ['HOME'], 'Documents', 'Data', 'sdss', 'dr16', 'sdss', 'spectro', 'redux')
run2d = '26'
plate = '2955'
mjd = '54562'
sdss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
sdss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spZbest-{plate}-{mjd}.fits')
print(sdss_spectra)
print(sdss_redshifts)

In [ ]:
run2d = 'v5_13_0'
plate = '9599'
mjd = '58131'
eboss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
eboss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, run2d, f'spZbest-{plate}-{mjd}.fits')
print(eboss_spectra)
print(eboss_redshifts)

## DESI spectra

Read the spectra and corresponding wavelength solutions.  Multiple exposures for each spectra may exist, so we do a simple coadd to get one spectrum per object.  We'll need to clean up some anomalous values first.

In [ ]:
desi_z = Table.read(desi_redshifts, 'ZBEST')
w = np.where(desi_z['TARGETID'] > 0)[0][0:50]
desi_z = desi_z[w]
desi = read_spectra(desi_spectra, single=True, coadd=desi_z['TARGETID'])
desi

In [ ]:
plotspectra(desi, zcatalog=desi_z, model_from_zcat=True,
            notebook=True, title=os.path.basename(desi_spectra),
            with_coaddcam=False, with_thumb_tab=False)

## SDSS Spectra

In [ ]:
sdss = read_spPlate(sdss_spectra, limit=50)
sdss_z, sdss_model = read_spZbest(sdss_redshifts, limit=50)
sdss


In [ ]:
plotspectra(sdss, zcatalog=sdss_z, model=(sdss_model.spectral_axis.value, sdss_model.flux.value),
            notebook=True, title=os.path.basename(sdss_spectra),
            model_from_zcat=False, with_coaddcam=False, mask_type='PRIMTARGET', with_thumb_tab=False)


## eBOSS Spectra

In [ ]:
eboss = read_spPlate(eboss_spectra, limit=50)
eboss_z, eboss_model = read_spZbest(eboss_redshifts, limit=50)
eboss


In [ ]:
plotspectra(eboss, zcatalog=eboss_z, model=(eboss_model.spectral_axis.value, eboss_model.flux.value),
            notebook=True, title=os.path.basename(eboss_spectra),
            model_from_zcat=False, with_coaddcam=False, mask_type='EBOSS_TARGET1', with_thumb_tab=False)

